In [1]:
from pymongo import MongoClient
import pandas as pd
from dotenv import load_dotenv
import os

# Load variables from .env
load_dotenv()

# Get the MongoDB URI
conn_str = os.getenv("MONGO_SRV")

In [2]:
def load_data():
    # Connect to MongoDB
    client = MongoClient(conn_str)
    db = client["test"]          # Replace with your database name
    collection = db["responses"] 
# Load all documents into a DataFrame
    data = pd.DataFrame(list(collection.find()))
    return data

def calculate_overall_correct(data):
    # Add a column to check if the answer is correct
    data["is_correct"] = (data["selection"] == data["trueLabel"])

    # Group by prolificId and count correct answers
    correct_counts = data[data["is_correct"]].groupby("prolificId").size().reset_index(name="correct_count")

    return correct_counts

In [3]:
data = load_data()

In [4]:
data[data['rt'].isna()].prolificId.unique()

array(['674b35aef9b2ae3dcdd26213', 'unknown'], dtype=object)

In [5]:
correct = calculate_overall_correct(data)

In [6]:
correct_by_trial = (
    data[data["is_correct"]]
    .groupby("trialType")
    .size()
    .reset_index(name="correct_count")
)
print(correct_by_trial)

               trialType  correct_count
0       motion_direction           1312
1  object_identification           1670


In [7]:
accuracy_by_trial = (
    data.groupby("trialType")["is_correct"]
    .mean()
    .reset_index(name="accuracy")
)
print(accuracy_by_trial)

               trialType  accuracy
0       motion_direction  0.764123
1  object_identification  0.970366


In [8]:
user_stats = (
    data.groupby("prolificId")
    .agg(
        total_responses=("is_correct", "count"),
        correct_answers=("is_correct", "sum"),
        accuracy=("is_correct", "mean")
    )
    .reset_index()
)
print(user_stats)

                  prolificId  total_responses  correct_answers  accuracy
0   5acb7b0ae1099600016acdd6               80               69  0.862500
1   5be12624b08fb60001f5680a               80               71  0.887500
2   5cc76f310c352e00161975b4               80               74  0.925000
3   60b0ba56f6a2244bbae05690               80               68  0.850000
4   60f69afe109d7b0fa70bd044               80               65  0.812500
5   647c65c5acb91c89819c7970               80               71  0.887500
6   64b4c078aeb3a03a356ce3b2               80               72  0.900000
7   6511f1f1490c9df9f8c7890e               80               64  0.800000
8   6522a42f39b5bd8f96735aa9               80               68  0.850000
9   655652e4d1acd4df9b343078               79               64  0.810127
10  65847ecf0896ca006d2c00b7               80               74  0.925000
11  65a69ce9c8cf8262d34f646a               80               72  0.900000
12  65fade256d70be4d6ea4ecaa               78      

In [9]:
user_stats_by_type = (
    data.groupby(["prolificId", "trialType"])
    .agg(
        total_responses=("is_correct", "count"),
        correct_answers=("is_correct", "sum"),
        accuracy=("is_correct", "mean")
    )
    .reset_index()
)
print(user_stats_by_type)

                  prolificId              trialType  total_responses  \
0   5acb7b0ae1099600016acdd6       motion_direction               40   
1   5acb7b0ae1099600016acdd6  object_identification               40   
2   5be12624b08fb60001f5680a       motion_direction               40   
3   5be12624b08fb60001f5680a  object_identification               40   
4   5cc76f310c352e00161975b4       motion_direction               40   
..                       ...                    ...              ...   
86  67f2b1f27c9c5092a7ac8c84  object_identification               40   
87  67f6784e63c8958eba79d3d9       motion_direction               40   
88  67f6784e63c8958eba79d3d9  object_identification               40   
89                   unknown       motion_direction                2   
90                   unknown  object_identification                3   

    correct_answers  accuracy  
0                29     0.725  
1                40     1.000  
2                31     0.775  
3      

In [12]:
# Remove rows with missing rt
clean_data = data.dropna(subset=["rt"])
clean_data['rt'] = clean_data['rt'].astype(int)

# Now group and compute stats
avg_rt_by_user_type = (
    clean_data.groupby(["prolificId", "trialType"])
    .agg(avg_rt=("rt", "mean"))
    .reset_index()
)

print(avg_rt_by_user_type)


                  prolificId              trialType    avg_rt
0   5acb7b0ae1099600016acdd6       motion_direction  6720.650
1   5acb7b0ae1099600016acdd6  object_identification  3616.925
2   5be12624b08fb60001f5680a       motion_direction  3866.350
3   5be12624b08fb60001f5680a  object_identification  6803.325
4   5cc76f310c352e00161975b4       motion_direction  2788.200
..                       ...                    ...       ...
83  67f2b1f27c9c5092a7ac8c84       motion_direction  4860.975
84  67f2b1f27c9c5092a7ac8c84  object_identification  2233.925
85  67f6784e63c8958eba79d3d9       motion_direction  6011.675
86  67f6784e63c8958eba79d3d9  object_identification  7764.150
87                   unknown  object_identification  2479.000

[88 rows x 3 columns]


/var/folders/mq/p952ryqs1gv79cnvz9f05slr0000gn/T/ipykernel_12742/1311787591.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data['rt'] = clean_data['rt'].astype(int)


In [13]:
avg_rt_by_user_type

,prolificId,trialType,avg_rt
0,5acb7b0ae1099600016acdd6,motion_direction,6720.650
1,5acb7b0ae1099600016acdd6,object_identification,3616.925
2,5be12624b08fb60001f5680a,motion_direction,3866.350
3,5be12624b08fb60001f5680a,object_identification,6803.325
4,5cc76f310c352e00161975b4,motion_direction,2788.200
...,...,...,...
83,67f2b1f27c9c5092a7ac8c84,motion_direction,4860.975
84,67f2b1f27c9c5092a7ac8c84,object_identification,2233.925
85,67f6784e63c8958eba79d3d9,motion_direction,6011.675
86,67f6784e63c8958eba79d3d9,object_identification,7764.150
